<a href="https://colab.research.google.com/github/gabordun/Python_workshops/blob/master/import_from_web3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

#import requests
from urllib import request
import requests
import time

basic=requests.get("https://index.hu")
soup=BeautifulSoup(basic.text,'html.parser')
     
table = {'Description': [], 'Location':[] , 'Cost':[] }
for item in soup.table.find_all('tr'):
 if item.th.text == 'Description':
   table[item.th.text].append(item.td.text.replace('\n',''))
 if item.th.text == 'Location':
   table[item.th.text].append(item.td.text)
 if item.th.text == 'Cost':
   table[item.th.text].append(item.td.text)
df = pd.DataFrame( data = table)
df

AttributeError: ignored

In [39]:
# read source html
import requests
r = requests.get("https://en.wikipedia.org/wiki/Formula_One")
doc = BeautifulSoup(r.text, "html.parser")

new_table = pd.DataFrame(columns=range(0,100), index = [1]) # I know the size 

row_marker = 0
for row in doc.find_all('tr'):
  column_marker = 0
  columns = row.find_all('td')
  for column in columns:
    new_table.iat[row_marker,column_marker] = column.get_text()
    column_marker += 1
    
new_table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
1,\nBNF: cb12002946q (data)\nGND: 4267570-4\nLCC...,\nAdventure racing\nAmerican football\nmen\nwo...,\nAir sports\nBallooning\nGliding\nParachuting...,\nLeague of Legends\nDota 2\nCounter-Strike\nS...,\n\n1980 Williams\n1981 Williams\n1982 Ferr...,\n\n1990 McLaren\n1991 McLaren\n1992 Willia...,\n\n2000 Ferrari\n2001 Ferrari\n2002 Ferrar...,\n\n2010 Red Bull\n2011 Red Bull\n2012 Red ...,\n\n2010 Red Bull\n2011 Red Bull\n2012 Red ...,\n3. Daniel Ricciardo\n31. Esteban Ocon\n,\n6. Nicholas Latifi\n63. George Russell\n,\n6. Nicholas Latifi\n63. George Russell\n,\n4. Lando Norris\n55. Carlos Sainz Jr.\n,MercedesRacing Point-BWT MercedesRed Bull Raci...,MercedesRacing Point-BWT MercedesRed Bull Raci...,Mercedes,Racing Point-BWT Mercedes,Red Bull Racing-Honda,Renault,Williams-Mercedes,\n44. Lewis Hamilton\n77. Valtteri Bottas\n,\n11. Sergio Pérez\n18. Lance Stroll\n,\n23. Alexander Albon\n33. Max Verstappen\n,\n3. Daniel Ricciardo\n31. Esteban Ocon\n,\n6. Nicholas Latifi\n63. George Russell\n,\n1950\n1951\n1952\n1953\n1954\n1955\n1956\n19...,\nAustralian\nBahrain\nVietnamese\nChinese\nDu...,\nArgentine\nCaesars Palace\nDallas\nDetroit\n...,\nMiami\n,– Grands Prix affected by the coronavirus pan...,\nAustin\nBahrain\nBaku\nBarcelona\nHanoi\nHun...,\nAdelaide\nAida\nAin-Diab\nAintree\nAnderstor...,\nAlfa Romeo\nAlphaTauri\nFerrari\nHaas\nMcLar...,\nAston Martin (2021)\n,\nAFM\nAGS\nAlta\nAmon\nAndrea Moda\nApollon\n...,Although World Championship races held in 1952...,\nArgentina\nAustralia\nAustria\nBelgium\nBraz...,\n\n1950 G. Farina\n1951 J. M. Fangio\n19...,,\n\n1950 G. Farina\n1951 J. M. Fangio\n1952 ...,...,\n2019\n2020\n,\nList of FIA events\nList of FIA member organ...,\nF1\nF2\nF3\nF4\nFormula 1000\nFormula Atlant...,\nF500\nF3000\nF5000\nFormula A (SCCA)\nFormul...,\nCFGP\nFormula Car Challenge\nFormula Mazda\n...,\nA1GP\nADAC Formel Masters\nAuto GP\nBarber P...,\nKF1\nKF2\nKF3\nKZ1\nKZ2\nSuperkart\n,\nDTM\nWTCR\nBTCC\nGroup F\nGroup G\nGroup H\n...,\nAppendix J\nBTC-T\nGroup 1\nGroup 2\nGroup 5...,\nARCA\nAllison Legacy Series\nAUSCAR\nIMCA Sp...,\nBriSCA F1\nBriSCA F2\nV8 Hotstox\nHot Rods\n...,\nGroup R\nGroup R-GT\nSuper 2000\nSuper 1600\...,\nGroup 1\nGroup 2\nGroup 4\nGroup A\nGroup B\...,\nDPi\nLMP\nLMP1\nLMP2\nLMP3\n,\nClubmans\nDP\nGroup 3\nGroup 4\nGroup 5\nGro...,\nLM GTE (GT2)\nGT3\nGT4\nGT500\nGT300\nTrans-...,\nGroup 3\nGroup 4\nGroup 5\nGroup B\nGroup D ...,\nTop Fuel Dragster (TF/D)\nTop Alcohol Dragst...,\nA/FX (Factory Experimental)\nAltered\nTop Ga...,\nBaja Bug\nDune buggy\nRallycross\nAutocross\...,\nList of world sports championships\n,Team\nAssociation football\nmen\nmen's club\nw...,\nAssociation football\nmen\nmen's club\nwomen...,\nArchery\nAquatic sports\nAthletics\noutdoor\...,\nBasque pelota\nCricket\nTest\nODI (men)\nODI...,Team\nAmputee football\nBlind football\nCP foo...,\nAmputee football\nBlind football\nCP footbal...,\nArchery\nAthletics\nBadminton\nBobsleigh and...,\nGrappling\nKendo\nKickboxing\nMuaythai\nSamb...,\nCarom billiards\nThree-cushion\nindividual\n...,\nBackgammon\nBridge\nChess\nopen\nwomen\nteam...,Auto racing\nFormula One\nKarting\nRallying\nR...,\nFormula One\nKarting\nRallying\nRallycross\n...,\nEndurance\nEnduro\nSuperEnduro\nIce racing\n...,\nAeroplane sport\nAerobatic\nAerobatic GP\nPo...,Team\nAdventure racing\nAmerican football\nmen...,\nAdventure racing\nAmerican football\nmen\nwo...,\nAir sports\nBallooning\nGliding\nParachuting...,\nLeague of Legends\nDota 2\nCounter-Strike\nS...,\nSee also: {{Main world cups}}\n


In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
    
class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [15]:
hp = HTMLTableParser()
table = hp.parse_url("https://en.wikipedia.org/wiki/Formula_One")[0][1] # Grabbing the table from the tuple
table.head()

KeyError: ignored